In [1]:
%load_ext autoreload
%autoreload 2

import datajoint as dj
dj.config["enable_python_native_blobs"] = True
dj.config['nnfabrik.schema_name'] = "nnfabrik_v1_tuning"
schema = dj.schema("nnfabrik_v1_tuning")


import os
from os import listdir
from os.path import isfile, join


import torch
import numpy as np
import pickle 
import time
import matplotlib.pyplot as plt
import seaborn as sns

import nnfabrik
from nnfabrik import main, builder
from nnfabrik.main import *

import nnvision
from nnvision.tables.from_nnfabrik import TrainedModel
# from nnvision.tables.legacy.from_mei import TrainedEnsembleModel

Connecting pawelp@134.2.168.16:3306


In [2]:
Fabrikant()

fabrikant_name Name of the contributor that added this entry,full_name full name of the person,email e-mail address,affiliation conributor's affiliation (e.g. Sinz Lab),dj_username DataJoint username
pawelp,,ppierzc@gmail.com,sinzlab,pawelp


In [3]:
Fabrikant().insert1(dict(fabrikant_name='pawelp',
                         email="ppierzc@gmail.com",
                         affiliation='sinzlab',
                         dj_username="pawelp"))

DuplicateError: ("Duplicate entry 'pawelp' for key 'PRIMARY'", 'To ignore duplicate entries in insert, set skip_duplicates=True')

In [4]:
Seed().insert([
    {'seed':1000},
    {'seed':2000},
    {'seed':3000},
    {'seed':4000},
    {'seed':5000}
])
Seed()

DuplicateError: ("Duplicate entry '1000' for key 'PRIMARY'", 'To ignore duplicate entries in insert, set skip_duplicates=True')

In [5]:
trainer_fn = 'nnvision.training.nnvision_trainer'
trainer_config = {'max_iter': 100,
 'verbose': False,
 'lr_decay_steps': 3,
 'avg_loss': False,
 'patience': 3,
 'lr_init': 0.0042}


dataset_fn = 'nnvision.datasets.monkey_static_loader'
dataset_config = {'dataset': 'CSRF19_V1',
 'neuronal_data_files': ['/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3639492658943.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3639664527524.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3639749909659.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3632669014376.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3632932714885.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3640095265572.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3639060843972.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3634658447291.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3636034866307.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3638973674012.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3634744023164.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3634142311627.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3637333931598.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3638541006102.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3633364677437.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3640011636703.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3634055946316.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3636552742293.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3635178040531.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3635949043110.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3638367026975.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3637161140869.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3637851724731.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3638802601378.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3638885582960.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3638373332053.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3637248451650.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3637760318484.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3639406161189.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3638456653849.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3631896544452.pickle',
  '/data/monkey/toliaslab/CSRF19_V1/neuronal_data/CSRF19_V1_3631807112901.pickle'],
 'image_file': '/data/monkey/toliaslab/CSRF19_V1/images/CSRF19_V1_images.pickle',
 'image_cache_path': '/data/monkey/toliaslab/CSRF19_V1/images/individual',
 'crop': 96,
 'subsample': 1,
 'seed': 1000,
 'time_bins_sum': 12,
 'batch_size': 128}

model_fn = 'nnvision.models.se_core_point_readout'
model_config = {'pad_input': False,
 'stack': -1,
 'depth_separable': True,
 'input_kern': 24,
 'gamma_input': 10,
 'gamma_readout': 0.5,
 'hidden_dilation': 2,
 'hidden_kern': 9,
 'se_reduction': 16,
 'n_se_blocks': 2,
 'hidden_channels': 32}

In [6]:
Dataset().add_entry(dataset_fn, dataset_config, dataset_comment='CSRF19_V1')
Dataset()

ValueError: Corresponding entry already exists

In [7]:
Model().add_entry(model_fn, model_config, model_comment='isotropic')
Model()

ValueError: Corresponding entry already exists

In [8]:
Trainer().add_entry(trainer_fn, trainer_config, trainer_comment="max_iter: 100")
Trainer()

trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,trainer_config training configuration object,trainer_fabrikant Name of the contributor that added this entry,trainer_comment short description,trainer_ts UTZ timestamp at time of insertion
nnvision.training.nnvision_trainer,2a2816c50c67295376b4e89448430985,=BLOB=,pawelp,max_iter: 100,2020-11-21 14:51:33


In [39]:
TrainedModel.populate({'seed': '1000'}, reserve_jobs=True)

Epoch 1: 100%|██████████| 3616/3616 [00:56<00:00, 63.57it/s]


[001|00/03] ---> 0.26950716972351074


Epoch 2: 100%|██████████| 3616/3616 [00:49<00:00, 72.59it/s]


[002|00/03] ---> 0.3319963812828064


Epoch 3: 100%|██████████| 3616/3616 [00:51<00:00, 70.18it/s]


...

[051|01/03] -/-> 0.3985925614833832


Epoch 52: 100%|██████████| 3616/3616 [00:49<00:00, 72.89it/s]


[052|02/03] -/-> 0.3987319767475128


Epoch 53: 100%|██████████| 3616/3616 [00:51<00:00, 70.69it/s]


[053|03/03] -/-> 0.39886999130249023
Restoring best model after lr decay! 0.398870 ---> 0.399311
Restoring best model! 0.399311 ---> 0.399311


In [2]:
TrainedModel()# & dict(model_hash = )

NameError: name 'TrainedModel' is not defined

In [35]:
from nnvision.tables.legacy.from_mei import TrainedEnsembleModel
TrainedEnsembleModel()

dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,ensemble_hash the hash of the ensemble
